# Veiwing the output of the BOOK NLP

In [2]:
import pandas as pd

# Load entities file
entities = pd.read_csv("/yunity/arusty/SoundScribe/book_files/lotl-chapter4.entities", sep="\t", header=0)

# Show first few rows
print(entities.head())


   COREF  start_token  end_token  prop  cat                     text
0      2            6          6  PROP  PER                   Sibier
1      2           11         11  PRON  PER                       he
2     14           14         16   NOM  PER      the nearby vagrants
3    108           21         23   NOM  PER  that slithering soldier
4    109           26         26  PRON  PER                      our


In [3]:
# Show only PERSON entities
people = entities[entities["cat"] == "PER"]
print(people)


     COREF  start_token  end_token  prop  cat                     text
0        2            6          6  PROP  PER                   Sibier
1        2           11         11  PRON  PER                       he
2       14           14         16   NOM  PER      the nearby vagrants
3      108           21         23   NOM  PER  that slithering soldier
4      109           26         26  PRON  PER                      our
..     ...          ...        ...   ...  ...                      ...
576    112         3186       3186   NOM  PER                      kid
577      4         3189       3189  PROP  PER                   Ilyian
578      4         3196       3196  PRON  PER                      his
579      2         3202       3202  PROP  PER                   Sibier
580    107         3206       3206  PRON  PER                     they

[569 rows x 6 columns]


# Creating text files

In [4]:
import pandas as pd
import re
import os

# File paths
original_text_file = "/yunity/arusty/SoundScribe/text_files/chapter4.txt"  # Replace with your actual input text file
book_title = "lotl-chapter4"  # Replace with your actual book title
output_dir = "book-text-info/" + book_title + "/" # Adjust if needed
audio_text_file = output_dir + book_title + "-book-audio-text.txt"
character_dialogue_file = output_dir + book_title + "-character-dialogue.txt"
quotes_file = "book_files/" + book_title + ".quotes"  # Replace with your actual quotes file
entities_file = "book_files/" + book_title + ".entities"  # Replace with your actual quotes file


In [ ]:
# Load quotes (Speaker is stored as ID)
quotes_df = pd.read_csv(quotes_file, sep="\t", header=0)

# Load entities to resolve character names
entities_df = pd.read_csv(entities_file, sep="\t")

# Filter only person entities (PER)
person_entities = entities_df[entities_df["cat"] == "PER"]

# Create a mapping of {start_token: character_name}
character_map = {}

for _, row in person_entities.iterrows():
    entity_id = str(row["start_token"])  # Ensure ID is a string
    character_name = row["text"]  # Get the character's name
    
    # If the ID was seen before with a pronoun, update to actual name
    if entity_id in character_map and character_name.islower():
        continue  # Avoid overwriting a proper name with a pronoun
    
    character_map[entity_id] = character_name

# Resolve speaker names from IDs
quotes_df["Resolved Speaker"] = quotes_df["char_id"].astype(str).map(character_map)

# --- File 1: Character Quotes List ---
character_quotes = {}

for _, row in quotes_df.iterrows():
    speaker = row["Resolved Speaker"] if pd.notna(row["Resolved Speaker"]) else "Unknown"
    quote = row["quote"]
    
    if speaker not in character_quotes:
        character_quotes[speaker] = []
    
    character_quotes[speaker].append(f'{quote} | ')

# Save Character Quotes List
os.makedirs(output_dir, exist_ok=True)
with open(character_dialogue_file, "w") as f:
    for character, quotes in character_quotes.items():
        f.write(f"{character}: {' '.join(quotes)}\n")

# --- File 2: Annotated Text ---
# Read original text
with open(original_text_file, "r") as f:
    text = f.read()

# Insert speaker names before each quote
for _, row in quotes_df.iterrows():
    speaker = row["Resolved Speaker"] if pd.notna(row["Resolved Speaker"]) else "Unknown"
    quote = row["quote"]
    
    # Create replacement format
    tagged_quote = f"</{speaker}> {quote}"
    
    # Replace only the first occurrence of the quote
    text = re.sub(re.escape(quote), tagged_quote, text, count=1)

# Save Annotated Text
with open(audio_text_file, "w") as f:
    f.write(text)

print("Files saved successfully!")

Files saved successfully!


In [7]:
import pandas as pd

# Load entities file
entities_file = "book_files/lotl-chapter4.entities"
entities_df = pd.read_csv(entities_file, sep="\t")

# Filter for person entities
person_entities = entities_df[entities_df["cat"] == "PER"]

# Create the mapping of character IDs to names
character_map = {}

for _, row in person_entities.iterrows():
    entity_id = str(row["COREF"])  # This might be 'COREF' instead of 'start_token'
    character_name = row["text"]
    
    if entity_id in character_map and character_name.islower():
        continue  # Avoid replacing proper names with pronouns
    
    character_map[entity_id] = character_name

# Print the mapping for verification
print(character_map)


{'2': 'Sibier', '14': 'the nearby vagrants', '108': 'that slithering soldier', '109': 'our', '15': 'the vagrant', '16': 'his partner', '3': 'His', '4': 'Ilyian', '18': 'his four - year - old brother', '19': 'his younger brother', '20': 'the soldier', '22': 'they', '24': 'They', '26': 'our', '27': 'His father', '23': 'the Kingdom', '29': 'the two vagrants', '31': 'He', '32': 'a soldier', '33': 'warriors', '34': 'anyone', '35': 'His', '36': 'his smaller companion', '37': 'the older man', '39': 'These men', '5': 'Terit', '38': 'the bigger vagrant', '40': 'men', '41': 'the mob on Earl ’s Road', '42': 'the king', '44': 'men like them who had voiced the unrest of the people', '45': 'the people', '48': 'the man', '49': 'yourself', '50': 'Their', '52': 'Their family', '6': 'Father', '53': 'the scrawny vagrant', '7': 'Yorin', '110': 'We', '54': 'them', '0': 'I', '56': 'his brother', '111': 'we', '57': 'His', '59': 'his family', '61': 'the distant figure of the captain of the seventh district', 

In [13]:
quotes_file = "book_files/lotl-chapter4.quotes"

# Read quotes
quotes_df = pd.read_csv(quotes_file, sep="\t", header=0)

# Print first few rows
quotes_df.head()


,quote_start,quote_end,mention_start,mention_end,mention_phrase,char_id,quote
0,18,33,34,35,the vagrant,15,“ Look at that slithering soldier eating in ou...
1,570,578,579,581,the bigger vagrant,38,"“ We should fight him , Terit , ”"
2,585,591,592,592,Sibier,2,“ Take our food back . ”
3,756,760,761,761,he,2,"“ I will , ”"
4,765,773,761,761,he,2,"“ I will do it , Father . ”"


In [14]:
print(character_map.get("15", "Unknown"))  # Should return a character name
print(character_map.get("38", "Unknown"))
print(character_map.get("2", "Unknown"))

the vagrant
the bigger vagrant
Sibier


In [18]:
import pandas as pd

# Load entities file
entities_file = "book_files/lotl-chapter4.entities"
entities_df = pd.read_csv(entities_file, sep="\t")

# Filter for person-related entities
person_entities = entities_df[entities_df["cat"] == "PER"].copy()

# Sort entities by token position to process in order of appearance
person_entities = person_entities.sort_values(by=["start_token"])

# Initialize mapping
coref_map = {}

for _, row in person_entities.iterrows():
    entity_id = str(row["COREF"])  # Coreference ID (e.g., "15")
    entity_text = row["text"]

    # If the ID is already mapped to a real name, update earlier mentions
    if entity_id in coref_map:
        if entity_text.islower():
            continue  # Avoid replacing real names with pronouns
    coref_map[entity_id] = entity_text  # Store latest reference

# Back-propagate known names to earlier mentions
resolved_map = {}
last_seen_name = {}

for entity_id, entity_text in coref_map.items():
    if entity_text.istitle():  # Proper names (Sibier, Yorin, etc.)
        last_seen_name[entity_id] = entity_text
    else:
        if entity_id in last_seen_name:  # Replace with last known proper name
            entity_text = last_seen_name[entity_id]
    
    resolved_map[entity_id] = entity_text  # Store final mapping

# Print fixed mappings
print(resolved_map)


{'2': 'Sibier', '14': 'the nearby vagrants', '108': 'that slithering soldier', '109': 'our', '15': 'the vagrant', '16': 'his partner', '3': 'His', '4': 'Ilyian', '18': 'his four - year - old brother', '19': 'his younger brother', '20': 'the soldier', '22': 'they', '24': 'They', '26': 'our', '27': 'His father', '23': 'the Kingdom', '29': 'the two vagrants', '31': 'He', '32': 'a soldier', '33': 'warriors', '34': 'anyone', '35': 'His', '36': 'his smaller companion', '37': 'the older man', '39': 'These men', '5': 'Terit', '38': 'the bigger vagrant', '40': 'men', '41': 'the mob on Earl ’s Road', '42': 'the king', '44': 'men like them who had voiced the unrest of the people', '45': 'the people', '48': 'the man', '49': 'yourself', '50': 'Their', '52': 'Their family', '6': 'Father', '53': 'the scrawny vagrant', '7': 'Yorin', '110': 'We', '54': 'them', '0': 'I', '56': 'his brother', '111': 'we', '57': 'His', '59': 'his family', '61': 'the distant figure of the captain of the seventh district', 

In [20]:
quotes_df["char_id"] = quotes_df["char_id"].astype(str)
quotes_df["Resolved Speaker"] = quotes_df["char_id"].map(resolved_map)

# Print any unmapped IDs
unmapped = quotes_df[quotes_df["Resolved Speaker"].isna()]
print("Unmapped Speaker IDs:", unmapped["char_id"].unique())

Unmapped Speaker IDs: []


In [50]:
# ----------------------
# Step 1: Build the Resolved Mapping from Entities
# ----------------------
# Load the entities file (assuming it's tab-separated)
entities_df = pd.read_csv(entities_file, sep="\t")

# Filter for person entities (where 'cat' equals "PER")
person_entities = entities_df[entities_df["cat"] == "PER"].copy()

# Sort entities by token position if needed (here using 'start_token')
person_entities = person_entities.sort_values(by=["start_token"])

# Build an initial mapping: use the 'COREF' column as the key and the 'text' column as the name.
# We convert the COREF value to a string.
initial_map = {}
for _, row in person_entities.iterrows():
    entity_id = str(row["COREF"])
    entity_text = row["text"]
    # Prefer proper names (if entity_text is title-cased) over pronouns or descriptions
    if entity_id in initial_map:
        if not entity_text.istitle():
            continue  # Skip if we already have a proper name for this ID
    initial_map[entity_id] = entity_text

# Optionally, you can print the initial mapping for debugging:
print("Initial Mapping:")
print(initial_map)

# Now, if you wish to backfill or override less informative labels with later, better ones,
# you can simply use the initial_map as your resolved mapping.
resolved_map = initial_map

# ----------------------
# Step 2: Load Quotes and Map Speaker IDs to Resolved Names
# ----------------------
# Load quotes file; assume it's tab-separated with two columns: Quote and char_id.
quotes_df = pd.read_csv(quotes_file, sep="\t", header=0)

# Convert char_id to string for proper matching
quotes_df["char_id"] = quotes_df["char_id"].astype(str)

# Map char_id to real character names using our resolved_map.
quotes_df["Resolved Speaker"] = quotes_df["char_id"].map(resolved_map)

# Optionally, check if there are any unmapped IDs (should be empty if all mapped correctly)
unmapped = quotes_df[quotes_df["Resolved Speaker"].isna()]
print("Unmapped Speaker IDs:", unmapped["char_id"].unique())

# ----------------------
# Step 3: Create File 1 – Character Dialogue File
# ----------------------
# Build a dictionary mapping each resolved speaker to a list of their quotes.
character_quotes = {}
for _, row in quotes_df.iterrows():
    speaker = row["Resolved Speaker"] if pd.notna(row["Resolved Speaker"]) else "Unknown"
    quote = row["quote"]
    if speaker not in character_quotes:
        character_quotes[speaker] = []
    character_quotes[speaker].append(f'{quote} | ')

# Ensure the output directory exists
os.makedirs(output_dir, exist_ok=True)

# Write the character dialogue file
with open(character_dialogue_file, "w", encoding="utf-8") as f:
    for character, quotes in character_quotes.items():
        f.write(f"{character}: {' '.join(quotes)}\n")

# Ensure that the quotes are processed in order of their starting offsets
quotes_df = quotes_df.sort_values(by="quote_start")

# Extract the list of resolved speakers and quotes
resolved_speakers = quotes_df["Resolved Speaker"].tolist()
quote_texts = quotes_df["quote"].tolist()

# For debugging: print the list of speakers
print("Resolved Speakers:", resolved_speakers)

# Read the original text
with open(original_text_file, "r", encoding="utf-8") as f:
    text = f.read()

# Define a regex pattern to match quotes (both straight and smart quotes)
pattern = r'(["“][^"”]+["”])'

# Create iterators for the resolved speakers and (optionally) the quote texts
speaker_iter = iter(resolved_speakers)
# If you want to compare the matched quote with the expected one, you can also use quote_iter:
# quote_iter = iter(quote_texts)

def replacer(match):
    # Get the next speaker from our iterator (if available)
    try:
        speaker = next(speaker_iter)
    except StopIteration:
        speaker = "Unknown"
    # Matched quote (including the surrounding quotes)
    matched_quote = match.group(0)
    # Return the speaker tag followed by the matched quote
    return f"</{speaker}> {matched_quote}"

# Use re.sub with our replacer function
annotated_text = re.sub(pattern, replacer, text, count=0)

# Save the annotated text to the output file
os.makedirs(os.path.dirname(audio_text_file), exist_ok=True)
with open(audio_text_file, "w", encoding="utf-8") as f:
    f.write(annotated_text)

print("Annotated text file saved successfully!")

Initial Mapping:
{'2': 'Sibier', '14': 'the nearby vagrants', '108': 'that slithering soldier', '109': 'our', '15': 'the vagrant', '16': 'his partner', '3': 'His', '4': 'Ilyian', '18': 'his four - year - old brother', '19': 'his younger brother', '20': 'the soldier', '22': 'they', '24': 'They', '26': 'our', '27': 'his father', '23': 'the Kingdom', '29': 'the two vagrants', '31': 'He', '32': 'a soldier', '33': 'warriors', '34': 'anyone', '35': 'His', '36': 'his smaller companion', '37': 'the older man', '39': 'We', '5': 'Terit', '38': 'the bigger vagrant', '40': 'men', '41': 'the mob on Earl ’s Road', '42': 'the king', '44': 'men like them who had voiced the unrest of the people', '45': 'the people', '48': 'the man', '49': 'yourself', '50': 'Their', '52': 'Their family', '6': 'Father', '53': 'the scrawny vagrant', '7': 'Yorin', '110': 'We', '54': 'them', '0': 'I', '56': 'his brother', '111': 'we', '57': 'His', '59': 'his family', '61': 'the distant figure of the captain of the seventh d

In [39]:
text

"\n\nOut of the Ashes</the vagrant> “ Look at that slithering soldier eating in our faces just to mock us , ”\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nSibier’s hair bristled as he listened to the nearby vagrants. \n“Look at that slithering soldier eating in our faces just to mock us,” the vagrant complained to his partner, his resentment tainting the air. Siber sniffed, squinting against the blistering wind that whipped in from the ocean. His stomach rumbled, twisting in hunger, but unable to consume the knot of hatred that boiled in him. Skies, he had lost everything. \nIlyian — his four-</the bigger va</Sibier> “ Take our food back . ” “ We should fight him , Terit , ” brother — tugged at Sibier’s shirt, letting out a low whine. Ilyian’s dark hair was matted like Sibier</Sibier></Sibier> “ I will do it , Father . ”l , ”but his face was so</Sibier> “ It ’s</Sibier> “ We either starve or die at the tip of his sword . ”Yorin , ”d rounder: evidence of the food Sibier had always shared wi

In [26]:
quotes_df

,quote_start,quote_end,mention_start,mention_end,mention_phrase,char_id,quote,Resolved Speaker
0,18,33,34,35,the vagrant,15,“ Look at that slithering soldier eating in ou...,the vagrant
1,570,578,579,581,the bigger vagrant,38,"“ We should fight him , Terit , ”",the bigger vagrant
2,585,591,592,592,Sibier,2,“ Take our food back . ”,Sibier
3,756,760,761,761,he,2,"“ I will , ”",Sibier
4,765,773,761,761,he,2,"“ I will do it , Father . ”",Sibier
5,826,833,834,834,he,2,"“ It ’s pointless , Yorin , ”",Sibier
6,843,856,834,834,he,2,“ We either starve or die at the tip of his sw...,Sibier
7,980,989,968,968,Sibier,2,“ Do you think we ’ll survive this ? ”,Sibier
8,1019,1024,1012,1012,his,57,“ Where ’s Ma ? ”,His
9,1288,1297,1283,1283,Sibier,2,“ I ’ll get you both some food . ”,Sibier


# Read the book file

In [47]:
import json

with open("/yunity/arusty/SoundScribe/book_files/lotl-chapter4.book", "r", encoding="utf-8") as f:
    book_data = json.load(f)

pd.DataFrame(book_data)

,characters
0,"{'agent': [{'w': 'listened', 'i': 12}, {'w': '..."
1,"{'agent': [{'w': 'spat', 'i': 816}, {'w': 'exc..."
2,"{'agent': [{'w': 'stood', 'i': 1805}, {'w': 'c..."
3,"{'agent': [{'w': 'pressed', 'i': 303}, {'w': '..."
4,"{'agent': [{'w': 'eaten', 'i': 1005}, {'w': 's..."
5,"{'agent': [{'w': 'allowed', 'i': 1624}, {'w': ..."
6,"{'agent': [{'w': 'taught', 'i': 363}, {'w': 'g..."
7,"{'agent': [{'w': 'narrowed', 'i': 1299}, {'w':..."
8,"{'agent': [{'w': 'saved', 'i': 2620}, {'w': 'l..."
9,"{'agent': [{'w': 'let', 'i': 1174}, {'w': 'bre..."
